In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from toxifate import *
import pycytominer as pcm
import deepod as dp

# start 01/08/2023, work in progress
# following the trend of top features (high STD) along the concentration gradient in Cell Painting data


pd.options.mode.chained_assignment = None  # default='warn'

# read in Cell Painting data
path = r"C:\Users\Roman\OneDrive - National University of Ireland, Galway\CellPainting project data\CellProfiler\PerWell\tubesPolyByWellProfiler.csv"
cp_df = pd.read_csv(path).drop(['Unnamed: 0','Count','Row','Column'], axis=1, errors='ignore')
# drop inf and nan values
cp_df = cp_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1, how="any")

if 'Concentration' in cp_df.columns:
    file_type = 'harmony'
    conc_col = 'Concentration'
    print('Harmony data')
elif 'Metadata_Concentration (Image)' in cp_df.columns:
    file_type = 'cellprofiler'
    conc_col = 'Metadata_Concentration (Image)'
    print('Cell Profiler data')

feature_categories = cp_feature_classifier(cp_df, file_type)
cp_df = drop_low_variance(cp_df, file_type='cellprofiler', threshold=cp_df.var(numeric_only=True).quantile(0.1))
cp_df.set_index(['PlateID', 'Metadata_Well', 'Metadata_Concentration (Image)', 'Compound'], inplace=True)
cp_df

Cell Profiler data
Low variance filter : 98 features dropped (9.899%)


AreaShape_Area (Nuclei)  \
PlateID   Metadata_Well Metadata_Concentration (Image) Compound                            
tubes1BR1 C10           30                             NCAP                    -0.788746   
          C11           100                            NCAP                     0.753405   
          C12           100                            NCAP                     0.001606   
          C13           300                            NCAP                    -1.559822   
          C14           300                            NCAP                    -0.364655   
...                                                                                  ...   
tubes5BR3 N5            0                              DMSO                     0.349177   
          N6            0                              DMSO                     1.334467   
          N7            10                             ZIDO                     1.827112   
          N8            10                             ZIDO                     1.588735   
          N9            30                             ZIDO                     1.620519   

                                                                 AreaShape_Compactness (Nuclei)  \
PlateID   Metadata_Well Metadata_Concentration (Image) Compound                                   
tubes1BR1 C10           30                             NCAP                           -0.998152   
          C11           100                            NCAP                           -0.207761   
          C12           100                            NCAP                            0.003045   
          C13           300                            NCAP                            0.424794   
          C14           300                            NCAP                            1.102571   
...                                                                                         ...   
tubes5BR3 N5            0                              DMSO                           -0.582309   
          N6            0                              DMSO                           -0.395801   
          N7            10                             ZIDO                           -0.192392   
          N8            10                             ZIDO                            0.746067   
          N9            30                             ZIDO                           -0.159868   

                                                                 AreaShape_ConvexArea (Nuclei)  \
PlateID   Metadata_Well Metadata_Concentration (Image) Compound                                  
tubes1BR1 C10           30                             NCAP                          -0.878957   
          C11           100                            NCAP                           0.581756   
          C12           100                            NCAP                          -0.044264   
          C13           300                            NCAP                          -1.542918   
          C14           300                            NCAP                          -0.366759   
...                                                                                        ...   
tubes5BR3 N5            0                              DMSO                           0.208182   
          N6            0                              DMSO                           1.457276   
          N7            10                             ZIDO                           1.863232   
          N8            10                             ZIDO                           1.550958   
          N9            30                             ZIDO                           1.707095   

                                                                 AreaShape_Eccentricity (Nuclei)  \
PlateID   Metadata_Well Metadata_Concentration (Image) Compound                                    
tubes1BR1 C10           30                             NCAP                            -1.044486   
     

In [3]:
from deepod.models.dsvdd import DeepSVDD
cp_df = cp_df.reset_index()
clf = DeepSVDD()
clf.fit(cp_df, y=None)
scores = clf.decision_function(cp_df)

Start Training...
ensemble size: 1


KeyError: 1038